In [ ]:
#!/usr/bin/env sage
import requests
import shutil
import gzip
import os

#################################################################
#
#  FullNut a program for handling nut fullerenes
#  Version 1.01
#  Date:   July 7, 2022
#
#  Author: Tomaž Pisanski
#          University of Primorska
#          FAMNIT DIST LATER
#################################################################



#from sage.all import *

#############################################################
# downloading and reading graph6 files provided by Nino Bašić
#############################################################

def download_remote_file(url):
	local_filename = url.split('/')[-1]
	if os.path.isfile(local_filename):
		return local_filename  # Already downloaded
	r = requests.get(url, stream=True)
	with open(local_filename, 'wb') as f:
		shutil.copyfileobj(r.raw, f)
	return local_filename

def read_graph6(filename):
	if filename.endswith('.gz'):
		with gzip.open(filename, 'rt') as f:
			for line in f:
				yield line.rstrip('\n')
	else:
		with open(filename, 'r') as f:
			for line in f:
				yield line.rstrip('\n')

#####################################
# one may either store latex or tikz
#####################################

def tex_header(docclass="standalone"):
    head = r'\documentclass[border=2mm]{'+docclass+"}\n"
    head +=r"\usepackage{tikz}"+"\n"
    head +=r"\begin{document}"+"\n"
    return head

def tikz_header(scale="0.05",sep = "0.7pt"):
    head = r'\begin{tikzpicture}[line width=0.5pt,fill opacity=0.9,'
    head +="scale = "+scale+"]\n"
    head +=r"\tikzstyle{every path}=[draw, thick]"+"\n"
    head +=r"\tikzstyle{every node}=[draw, circle, fill=red, inner sep="
    head += sep +"]\n"
    return head

tikz_footer =  r'''\end{tikzpicture}
'''
tex_footer = r'''\end{document}
'''


#######################################
# Input file provided by Jan Goedgebeur
# (in the same folder as the program)
#######################################
#ifname = "nut_fullerenes.g6"
###################################
#output folder
###################################
#ofolder = '/Users/tomopisanski1/Dropbox/graphs_latex/fullnuts/'    
#ofolder = 'fullnuts/schlegel'    


###########################
# drawing programs for graphs
###########################
def mydrawgraph(fig, fig_name,folder = "",docclass="standalone",scale="3.5",sep="0.4pt"):
    """Writes the latex/tikz code on a file."""
    f = open(folder + fig_name + '.tex', 'w')
    f.write(my_graph(fig,docclass=docclass,scale=scale,sep=sep))
    f.close()
    

def my_graph(g,docclass="standalone",scale="0.05",sep="0.7pt"):
    """Produces complete latex/tikz code for displaying 
    graph g."""
    ret = ""
    ret += tex_header(docclass)
    ret += tikz_header(scale=scale,sep=sep)
    for v, coord in g.layout().items():
        coord = float(coord[0]), float(coord[1])
        ret += r'\coordinate (v_{0}) at ({1:.5f}, {2:.5f}) {{}};'.format(v, coord[0], coord[1]) + '\n'
    enu = {}
    for u, v, label in g.edges():
        ret += r'\path (v_{0}) -- (v_{1});'.format(u, v) + '\n'
    if True:
        for v, coord in g.layout().items():
            color = "red"
            ret += r'\node[{2}] (x_{0}) at (v_{1}) {{}};'.format(v, v,color) + '\n'
    ret += tikz_footer
    ret += tex_footer
    return ret
    


###########################
# drawing programs for fullerenes
###########################
def mydrawlatex(fig, fig_name,folder = "",flcc = [(5,"gray")],iout=0,nts=None,save_latex=False,docclass="standalone",scale="3.5",sep="0.4pt"):
    """Writes the latex/tikz code on a file."""
    f = open(folder + fig_name + '.tex', 'w')
    f.write(my_latex(fig,flcc=flcc,nts=nts,docclass=docclass,scale=scale,sep=sep))
    f.close()
    

def my_latex(g,flcc = [(5,"gray")],nts=None,docclass="standalone",scale="0.05",sep="0.7pt"):
    """Produces complete latex/tikz code for displaying 
    graph/schlegel diagram g."""
    ret = ""
    ret += tex_header(docclass)
    ret += tikz_header(scale=scale,sep=sep)
    for v, coord in g.layout().items():
        coord = float(coord[0]), float(coord[1])
        ret += r'\coordinate (v_{0}) at ({1:.5f}, {2:.5f}) {{}};'.format(v, coord[0], coord[1]) + '\n'
    enu = {}
    for i,face in enumerate(g.faces()):
        for (fl,fc) in flcc:
            if len(face) == fl:
                if fl in enu:
                    fnum = enu[fl]
                else:
                    fnum = 1
                enu[fl] = fnum + 1 
                vface = toVface(face)
                pos = g.get_pos()
                (apx,flabel) = apex(vface,pos,fnum)
                apx = (float(apx[0]),float(apx[1]))
                ret += r'\fill[{0}]'.format(fc)
                for u,v in face:
                    ret += r'(v_{0}) -- '.format(u)
                ret += r'(v_{0});'.format(v)+"\n"
    for u, v, label in g.edges():
        ret += r'\path (v_{0}) -- (v_{1});'.format(u, v) + '\n'
    if nts:
        for v, coord in g.layout().items():
            color = "black"
            if abs(nts[v]) == 1:
                color = "red"
            elif abs(nts[v]) == 2:
                color = "blue"
            ret += r'\node[{2}] (x_{0}) at (v_{1}) {{}};'.format(v, v,color) + '\n'
    ret += tikz_footer
    ret += tex_footer
    return ret
    

#############################
# draws schlegel diagram of a fullerene with a hexagonal outer face
#############################
        
def schlegel_fullerene(g,nsteps = 100,flfc=[(5,"grey")],R=2.0,
                       phi=0.0,f=1.0,vertex_labels=False, verbose=False):
    """g -- fullerene graph
    nsteps -- numbr of iterations for spring embedder
    flfc -- face sizes and their color (only 5 and grey used here)
    R -- radius of circumscribed outer face
    phi -- offset angle for outer face
    vertex_labels -- display vertex labels
    verbose -- for testing purposes
    
    Returns
    g -- fullerene embedded graph
    oface -- outer face
    plt - graphic output"""
    g.is_planar(set_embedding=True)
    for face in g.faces():
        oface = toVface(face)
        k = len(oface)
        if k == 6:
            break
    ps = {}
    rphi = float(pi*phi)/float(360)
    verts = g.vertices()
    for v in verts:
        ps[v] = (0.,0.)
    for i,v in enumerate(oface):
        ps[v] = (R*cos(rphi+2*pi*i/float(k)),R*sin(rphi+2*pi*i/float(k)))
    for t in range(nsteps):
        for v in verts:
            if v not in oface:
                nbs = g.neighbors(v)
                nnb = len(nbs)
                avgx = avgy = 0
                for u in nbs:
                    (xu,yu) = ps[u] 
                    avgx = avgx+xu
                    avgy = avgy+yu
                avgx = float(avgx)/float(nnb)
                avgy = float(avgy)/float(nnb)
                ps[v] = (f*avgx,f*avgy)
    g.set_pos(ps)    
    plt = g.plot(vertex_labels=vertex_labels,vertex_size=10)
    for i,face in enumerate(g.faces()):
        vface = toVface(face)
        fal = len(vface)
        for (fl,fc) in flfc:
            if fal == fl:
                plt += polygon((ps[v] for v in vface),color = fc,axes=False)
    if verbose:
        plt.show()
    return(g,oface,plt)
        
###################################################
# Reading graphs written on a file in graph6 format
###################################################
        
def read_nut_fullerenes(fname = "nut_fullerenes.g6"):
    for g in read_graph6(fname):
        gr = Graph(g)
        yield gr

################################################
# Auxilliary routines
################################################

        
def toVface(face):
    """Face given by a sequence of edges (the last vertex repeats the first one.
    vface is a sequence ov vertices only."""
    vface = []
    for u,v in face:
        vface.append(u)
    return vface

def apex(face,pos,facenumber):
    '''Computes the center of gravity of a polygonal face.'''
    m = len(face)
    facelabel = "$"+str(m)+"_"+str(facenumber)+"$"
    pairs = [pos[v] for v in face]
    res = (sum([x for x,y in pairs])/m, sum([y for x,y in pairs])/m)
    return (res,facelabel)

################################################
#  Computations concerning nut graphs
################################################
def nut(gr):
    """ For a nut graph gr return its standard kernel eigenvector."""
    mat = gr.adjacency_matrix().kernel().basis()
    if len(mat) != 1:
        return False
    return standard(mat[0])
        
def standard(vec):
    """For a rational vector vec compute its standard integer form. """
    k = 1
    for a in vec:
        m = a.denominator()
        k = lcm(k,m)
    res = [k*a for a in vec]
    kk = gcd(res)
    res1 = [a/kk for a in res]
    return res1

def imagenut(g):
    """For a nut graph compute its image, i.e. the collection of sorted
    kernel eigenvector entries."""
    ng = nut(g)
    if not ng:
        return False
    vec = standard(ng)
    res = list(set(vec))
    nres = [-x for x in res]
    res.sort()
    nres.sort()
    if nres < res:
        res = nres[:]
    return res

################################################
#  Computations concerning pentagonal patches and 
#  automorphisms of a fullerene
################################################

def pentagons(ful):
    """For a given fullerene, and planar embedding determine all of its pentagons 
    (vertex lists)"""
    res = []
    for face in ful.faces():
        vface = toVface(face)
        if len(vface) == 5:
            res.append(vface)
    return res

####################################
# Checking how automorphism group of a fullerene
# acts on its pentagons.
# For future use only!!!!!!!!!!!!!
####################################

def pentagon_action(pentagons,a):
    """Determine the action of the fullerene automorphism a on the set of 
    12 pentagons."""
    spet = [set(x) for x in pentagons]
    rw = []
    for i,pent in enumerate(pentagons):
        ip = set([a(x) for x in pent])
        for j,p in enumerate(spet):
            if ip == p:
                rw.append(j)
                break
    return rw

def pentagon_autos(ful):
    """Determine the action of automorphism group of the fullerene ful on its
    12 pentagons."""
    ag = ful.automorphism_group()
    ful.is_planar(set_embedding=True)
    pents = pentagons(ful)
    res = []
    for a in ag:
        r = pentagon_action(pents,a)
        res.append(r)
    return res



####################################
# Pentagonal patches
####################################


def pentagonal_patches(ful):
    """Return graph composed of pentagonal patches of a fullerne """
    ful.is_planar(set_embedding=True)
    pos = ful.get_pos()
    edgs = []
    for face in ful.faces():
        if len(face)==5:
            edgs.extend(face)
    gr = Graph(edgs)
    gr.is_planar(set_embedding=True)
    return gr
    
####################################
# Inner duals for pentagonal patches
####################################
def pinner_dual(pentagons):
    """Return graph that is the inner dual restricted to pentagons.
    Isolated pentagons are ignored."""
    pents = [set(p) for p in pentagons]
    edgs = []
    for i,p in enumerate(pents):
        for j,q in enumerate(pents):
            if j <= i:
                continue
            if p.intersection(q) != set():
                edgs.append((i,j))
    pind = Graph(edgs)
    return pind

def pentagonal_inner_dual(ful):
    """Return graph composed of inner dual of pentagonal patches of a fullerne """
    ful.is_planar(set_embedding=True)
    pents = pentagons(ful)
    pind = pinner_dual(pents)
    return pind

####################################
# Partitions of 12 pentagons by pentagonal patches
####################################

def pentagonal_partition(ful):
    """For pentagonal partitions see 'Sizes of pentagonal clusters in fullerenes'
    by Nino Bašić, Gunnar Brinkmann, Patrick Fowler, Tomaž Pisanski and Nico Van Cleemput
    (J. Math. Chem. 55 (8), 2017, 1669-1682)"""
    p = pentagonal_inner_dual(ful)
    con = p.connected_components_sizes()
    con.sort()
    shc = con[::-1]
    res = sum(shc)
    k = 12-res
    shc += k*[1]
    res = str(shc[0])
    for p in shc[1:]:
        res += "+"+str(p)
    return res

#############################################
#
# Main program that reads all nut fullerenes 
# and stores all relevant information in memory
#
#############################################

    
    
def make_all_fullerenes(verbose=False):
    """Read nut fullerenes and prepare basic dictionaries
    fulls[n] ... n-th (fullerene,outer face)
    ptchs[ptc]... list of indices of fullerenes with a given ptc
    iduls[idu]... list of indices of fullerenes with a given idu
    parts[pt]... list of indices of fullerenes with a given pt
    """
    n = -1
    fulls = {} #fullerenes and outer faces
    ptchs = {} #patches
    iduls = {} #inner duals
    parts = {} #partitions
    images = {} #images
    for g in read_nut_fullerenes():
        n += 1
        gr1 = Graph(g)
        (gr,ofac,plt) = schlegel_fullerene(gr1)
        nts = nut(gr)
        img = str(imagenut(gr))
        fulls[n] = (gr,ofac,plt,nts,img)
        if img in images:
            images[img].append(n)
        else:
            images[img] = [n]
            if verbose:
                print(img)
        patches = pentagonal_patches(gr)
        isnew = True
        for pa in ptchs:
            if patches.is_isomorphic(pa):
                isnew = False
                ptchs[pa].append(n)
                break
        if isnew:
            px = patches.copy(immutable=True)
            ptchs[px] = [n]
            if verbose:
                gr.show(vertex_size=10,vertex_labels=None)
                plt.show()
                px.show(vertex_size=10,vertex_labels=None)
        innduls = pentagonal_inner_dual(gr)
        isnew = True
        for innd in iduls:
            if innduls.is_isomorphic(innd):
                isnew = False
                iduls[innd].append(n)
                break
        if isnew:
            idx = innduls.copy(immutable=True)
            iduls[idx] = [n]
            if  verbose:
                idx.show(vertex_size=10,vertex_labels=None)
        pts = pentagonal_partition(gr)
        isnew= True
        for par in parts:
            if par == pts:
                isnew = False
                parts[pts].append(n)
                break
        if isnew:
            parts[pts] = [n]
            if verbose:
                print(pts)
    print("number of fullerenes",len(fulls))
    print("number of non-isomorphic patches",len(ptchs))
    print("number of non-isomorphic inner duals",len(iduls))
    print("number of non-isomorphic partitions",len(parts))
    print("number of distinct images",len(images))
    return (fulls,ptchs,iduls,parts,images)


In [ ]:
#######################################################
# After this call all relevant information is available.
# Subsequent runs store basic statistic and latex/tikz images
# on varous files in subfolder outputs.
#######################################################

(fulls,ptchs,iduls,parts,images) = make_all_fullerenes()

In [ ]:
for k,gr in enumerate(iduls):
    mydrawgraph(gr,"pent_duals_"+str(k),folder = "outputs/duals/",docclass="standalone",scale="3.5",sep="3.4pt")


In [ ]:
for k,gr in enumerate(ptchs):
    mydrawgraph(gr,"pent_patches_"+str(k),folder = "outputs/patches/",docclass="standalone",scale="3.5",sep="2.4pt")


In [ ]:
ofolder = 'outputs/'    
with open(ofolder+"invariants.txt","w") as g:
    print()
    print("Invariants of 173 nut fullerenes")
    print()
    print("n - index")
    print("nv - number of vertices")
    print("edgs - list of edges")
    print("na - number of automorphisms")
    print("pts - pentagonal partition")
    print("img - image of kernel map")
    print("nts - kernel values on vertices")
    print()
    print("n;nv;edgs")
    print("n;na;pts;img;nts")
    print()
    for n in fulls:
        (gr,ofac,plt,nts,img) = fulls[n]
        eds = [(x,y) for (x,y,_) in gr.edges()]
        nv = gr.order()
        na = gr.automorphism_group().order()
        pts = pentagonal_partition(gr)
        print(n,nv,eds, sep = ";")
        print(n,na,pts,img,nts, sep= ";")
        print()
    print(file=g)
    print("Invariants of 173 nut fullerenes",file=g)
    print(file=g)
    print("n - index",file=g)
    print("nv - number of vertices",file=g)
    print("edgs - list of edges",file=g)
    print("na - number of automorphisms",file=g)
    print("pts - pentagonal partition",file=g)
    print("img - image of kernel map",file=g)
    print("nts - kernel values on vertices",file=g)
    print(file=g)
    print("n;nv;edgs",file=g)
    print("n;na;pts;img;nts",file=g)
    print(file=g)
    for n in fulls:
        (gr,ofac,plt,nts,img) = fulls[n]
        eds = [(x,y) for (x,y,_) in gr.edges()]
        nv = gr.order()
        na = gr.automorphism_group().order()
        pts = pentagonal_partition(gr)
        print(n,nv,eds, sep = ";",file=g)
        print(n,na,pts,img,nts, sep= ";",file=g)
        print(file=g)

        

In [ ]:
for k in fulls:
    gr = fulls[k][0]
    mydrawlatex(gr,"nut_ful_cc_"+str(k),folder = "outputs/fullerenes/",flcc = [(5,"gray")],
            nts=nut(gr),docclass="standalone",scale="3.5",sep="0.4pt")


In [ ]:
ofolder = 'outputs/'    
with open(ofolder+"images.txt","w") as g:
    print()
    print("Frequencies of images of nut fullerenes")
    print()
    print("Frequencies of images of nut fullerenes",file = g)
    print(file=g)
    for p in images:
        print(len(images[p]),":",p)
        print(len(images[p]),":",p,file=g)
    print()
    print("Images of nut fullerenes and corresponding indices")
    print()
    print(file=g)
    print("Images of nut fullerenes and corresponding indices",file = g)
    print(file=g)
    for p in images:
        print(images[p],":",p)
        print(images[p],":",p,file=g)


In [ ]:
ofolder = 'outputs/'    
with open(ofolder+"partitions.txt","w") as g:
    print()
    print("Frequencies of pentagonal partitions of nut fullerenes")
    print()
    print("Frequencies of pentagonal partitions of nut fullerenes",file = g)
    print(file=g)
    for p in parts:
        print(len(parts[p]),":",p)
        print(len(parts[p]),":",p,file=g)
    print()
    print("Pentagonal partitions of nut fullerenes and corresponding indices")
    print()
    print(file=g)
    print("Paentagonal partitions of nut fullerenes and corresponding indices",file = g)
    print(file=g)
    for p in parts:
        print(parts[p],":",p)
        print(parts[p],":",p,file=g)


In [ ]:
ofolder = 'outputs/'    
with open(ofolder+"inner_duals.txt","w") as g:
    print()
    print("Frequencies of pentagonal inner duals of nut fullerenes")
    print()
    print(file=g)
    print("Frequencies of pentagonal inner duals of nut fullerenes",file = g)
    print(file=g)
    for i,p in enumerate(iduls):
        print(i,len(iduls[p]))
    print()
    print("Inner dual indices of nut fullerenes and corresponding indices")
    print()
    for i,p in enumerate(iduls):
        print(i,iduls[p])
    for i,p in enumerate(iduls):
        print(i,len(iduls[p]),file = g)
    print(file=g)
    print("Inner dual indices of nut fullerenes and corresponding indices",file = g)
    print(file=g)
    for i,p in enumerate(iduls):
        print(i,iduls[p],file=g)



In [ ]:
ofolder = 'outputs/'    
with open(ofolder+"pathces.txt","w") as g:
    print()
    print("Frequencies of pentagonal pathces of nut fullerenes")
    print()
    print(file=g)
    print("Frequencies of pentagonal patches of nut fullerenes",file = g)
    print(file=g)
    for i,p in enumerate(ptchs):
        print(i,len(ptchs[p]))
    print()
    print("Pentagonal patch indices of nut fullerenes and corresponding indices")
    print()
    for i,p in enumerate(ptchs):
        print(i,ptchs[p])
    for i,p in enumerate(ptchs):
        print(i,len(ptchs[p]),file = g)
    print(file=g)
    print("Pentagonal patch indices of nut fullerenes and corresponding indices",file = g)
    print(file=g)
    for i,p in enumerate(ptchs):
        print(i,ptchs[p],file=g)

